In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/newexp/HC24/augmented_folder.zip -d /content/dataset/

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/augmented_folder/weighted/ST351_pred_bright.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST351_pred_cropped.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST351_pred_flipped.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST351_pred_original.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST351_pred_rotated.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST352_pred_bright.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST352_pred_cropped.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST352_pred_flipped.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST352_pred_original.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST352_pred_rotated.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST353_pred_bright.jpg  
  inflating: /content/dataset/augmented_folder/weighted/ST3

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, concatenate, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from glob import glob
import pandas as pd
import re  # Import regex module
import gc  # Add garbage collection
from itertools import cycle

# Set up Google Colab file saving
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for saving plots

# Create directory for saving plots in Colab
SAVE_DIR = '/content/densenet_results'
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"Results will be saved to: {SAVE_DIR}")

# Define paths to your segmentation results
segmentation_folders = ['max', 'average', 'confidence', 'vote', 'weighted']
base_path = '/content/dataset/augmented_folder/'  # Update this path

# Class mapping
class_mapping = {'FT': 0, 'ST': 1, 'TT': 2}
inv_class_mapping = {0: 'FT', 1: 'ST', 2: 'TT'}

# FIXED Function to load images and labels from a specific segmentation folder
def load_data_from_folder(seg_folder):
    """Load images and labels from a specific segmentation folder with enhanced error handling and duplicate prevention"""
    images = []
    labels = []
    filenames = []

    folder_path = os.path.join(base_path, seg_folder)

    if not os.path.exists(folder_path):
        print(f"❌ Folder does not exist: {folder_path}")
        return np.array([]), np.array([]), []

    # Get all image files using specific extensions
    image_extensions = ['*.png', '*.jpg', '*.jpeg', '*.PNG', '*.JPG', '*.JPEG']
    image_paths = []

    for ext in image_extensions:
        image_paths.extend(glob(os.path.join(folder_path, ext)))

    if not image_paths:
        print(f"❌ No image files found in: {folder_path}")
        print(f"   Checked extensions: {image_extensions}")
        return np.array([]), np.array([]), []

    # FIXED: Remove duplicates by converting to set and back to list
    image_paths = list(set(image_paths))
    image_paths.sort()  # Sort for consistent ordering

    print(f"🔄 Loading {len(image_paths)} unique images from {seg_folder} folder...")

    # Check a few sample filenames to understand the naming pattern
    sample_files = image_paths[:5]
    print(f"📋 Sample filenames:")
    for i, path in enumerate(sample_files):
        print(f"   {i+1}. {os.path.basename(path)}")

    successful_loads = 0
    failed_loads = 0
    processed_filenames = set()  # FIXED: Track processed filenames to avoid duplicates

    for img_path in image_paths:
        filename = os.path.basename(img_path)

        # FIXED: Skip if we've already processed this filename
        if filename in processed_filenames:
            print(f"⚠️  Skipping duplicate file: {filename}")
            continue

        processed_filenames.add(filename)

        # Extract class from filename (updated for new format like "FT1_pred_bright.png")
        # Try multiple regex patterns to handle different naming conventions
        patterns = [
            r'^(FT|ST|TT)\d*_pred_(?:bright|cropped|flipped|original|rotated)',  # Original pattern
            r'^(FT|ST|TT)\d*_.*',  # More flexible pattern
            r'^(FT|ST|TT)[\d_]',   # Even more flexible
            r'(FT|ST|TT)'          # Just find FT, ST, or TT anywhere
        ]

        class_name = None
        for pattern in patterns:
            class_match = re.search(pattern, filename)
            if class_match:
                class_name = class_match.group(1)
                break

        if not class_name:
            print(f"⚠️  Skipping file with unrecognized format: {filename}")
            failed_loads += 1
            continue

        if class_name not in class_mapping:
            print(f"⚠️  Skipping file with unknown class: {filename}")
            failed_loads += 1
            continue

        # Load and preprocess image
        try:
            img = cv2.imread(img_path)
            if img is None:
                print(f"⚠️  Warning: Could not read {img_path}")
                failed_loads += 1
                continue

            # Check image dimensions
            if img.shape[0] == 0 or img.shape[1] == 0:
                print(f"⚠️  Warning: Invalid image dimensions {img.shape} for {img_path}")
                failed_loads += 1
                continue

            # Resize to DenseNet121 input size (224x224)
            img = cv2.resize(img, (224, 224))
            # Convert BGR to RGB for DenseNet121
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            images.append(img)
            labels.append(class_mapping[class_name])
            filenames.append(filename)
            successful_loads += 1

        except Exception as e:
            print(f"❌ Error processing {img_path}: {e}")
            failed_loads += 1

    print(f"✅ Successfully loaded: {successful_loads} unique images")
    if failed_loads > 0:
        print(f"⚠️  Failed to load: {failed_loads} images")

    if successful_loads == 0:
        print(f"❌ No images could be loaded from {seg_folder}")
        return np.array([]), np.array([]), []

    # Print class distribution
    labels_array = np.array(labels)
    print(f"📊 Class distribution in {seg_folder}:")
    for class_name, class_id in class_mapping.items():
        count = np.sum(labels_array == class_id)
        print(f"   {class_name}: {count} images")

    # FIXED: Verify total count matches expected
    total_images = len(images)
    print(f"🔍 Total images loaded: {total_images}")

    # FIXED: Additional verification - check for any remaining duplicates
    unique_filenames = set(filenames)
    if len(unique_filenames) != len(filenames):
        print(f"⚠️  Warning: Found {len(filenames) - len(unique_filenames)} duplicate filenames after processing")
        # Remove duplicates while preserving order
        seen = set()
        unique_images = []
        unique_labels = []
        unique_files = []

        for img, label, fname in zip(images, labels, filenames):
            if fname not in seen:
                seen.add(fname)
                unique_images.append(img)
                unique_labels.append(label)
                unique_files.append(fname)

        images = unique_images
        labels = unique_labels
        filenames = unique_files
        print(f"✅ After deduplication: {len(images)} unique images")

    return np.array(images), np.array(labels), filenames


# MODIFIED model architecture using DenseNet121
def create_improved_densenet():
    # Load pre-trained DenseNet121 model without the top layer
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )

    # Freeze base model layers initially for transfer learning
    base_model.trainable = False

    # Add custom top layers
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(3, activation='softmax')  # 3 classes: FT, ST, TT
    ])

    return model

# Function to create fine-tuned DenseNet121
def create_finetuned_densenet():
    # Load pre-trained DenseNet121 model without the top layer
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )

    # Unfreeze the last few layers for fine-tuning
    base_model.trainable = True
    # Freeze all layers except the last 20
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    # Add custom top layers
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(3, activation='softmax')  # 3 classes: FT, ST, TT
    ])

    return model

# Function to plot and save ROC AUC curves
def plot_roc_auc_curve(y_true, y_pred_proba, seg_folder, save_dir=SAVE_DIR):
    """Plot and save multiclass ROC AUC curve"""
    n_classes = 3
    class_names = ['FT', 'ST', 'TT']

    # Binarize the output
    y_true_bin = label_binarize(y_true, classes=[0, 1, 2])

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_pred_proba.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(figsize=(10, 8))

    # Plot ROC curve for each class
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                label=f'ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})')

    # Plot micro and macro average ROC curves
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'Micro-average ROC curve (AUC = {roc_auc["micro"]:.2f})',
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label=f'Macro-average ROC curve (AUC = {roc_auc["macro"]:.2f})',
             color='navy', linestyle=':', linewidth=4)

    # Plot diagonal line
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC AUC Curves - {seg_folder} DenseNet121 Model')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)

    # Save the plot
    save_path = os.path.join(save_dir, f'roc_auc_curve_densenet_{seg_folder}.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"ROC AUC curve saved to: {save_path}")

    return roc_auc

# FIXED Function to train and evaluate a model for a specific segmentation method
def train_model_for_segmentation(seg_folder, epochs=20, fine_tune=True):
    print(f"\n=== Training DenseNet121 model for {seg_folder} segmentation ===")

    # Load data
    X, y, filenames = load_data_from_folder(seg_folder)

    if len(X) == 0:
        print(f"No valid images found in {seg_folder}. Skipping.")
        return None, None, None, None

    # FIXED: Verify data size before and after loading
    print(f"📊 Data verification for {seg_folder}:")
    print(f"   Total images loaded: {len(X)}")
    print(f"   Unique filenames: {len(set(filenames))}")
    print(f"   Expected ~7130 images per folder")

    if len(X) > 8000:  # If significantly more than expected
        print(f"⚠️  Warning: Loaded {len(X)} images, which seems high. Expected around 7130.")
        print(f"   This might indicate duplicate loading. Please verify folder contents.")

    # Split data with stratification to ensure balanced classes
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"📊 Data split for {seg_folder}:")
    print(f"   Training images: {len(X_train)}")
    print(f"   Testing images: {len(X_test)}")

    # MEMORY FIX: Convert to float32 and preprocess for DenseNet121
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # DenseNet121 preprocessing: normalize to [0,1] range
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Force garbage collection
    gc.collect()

    # Convert labels to categorical
    y_train_cat = to_categorical(y_train, num_classes=3)
    y_test_cat = to_categorical(y_test, num_classes=3)

    # Create DenseNet121 model
    if fine_tune:
        model = create_finetuned_densenet()
    else:
        model = create_improved_densenet()

    # Compile model with different learning rates for transfer learning
    if fine_tune:
        # Lower learning rate for fine-tuning
        model.compile(
            optimizer=Adam(learning_rate=0.00001),  # Lower LR for fine-tuning
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
    else:
        model.compile(
            optimizer=Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    # Create callbacks for better training
    model_checkpoint = ModelCheckpoint(
        os.path.join(SAVE_DIR, f'best_densenet_model_{seg_folder}.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,  # Increased patience for transfer learning
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,  # Increased patience
        min_lr=0.000001
    )

    # Train the model with appropriate batch size
    history = model.fit(
        X_train, y_train_cat,
        batch_size=16,  # Can use larger batch size with transfer learning
        epochs=epochs,
        validation_data=(X_test, y_test_cat),
        callbacks=[early_stopping, reduce_lr, model_checkpoint]
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(X_test, y_test_cat)
    print(f"{seg_folder} DenseNet121 model - Test accuracy: {test_acc:.4f}")

    # Get predictions
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Print classification report
    print(f"\nClassification Report for {seg_folder} DenseNet121 model:")
    print(classification_report(y_test, y_pred_classes,
                              target_names=list(inv_class_mapping.values())))

    # Plot and save confusion matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred_classes)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=list(inv_class_mapping.values()),
               yticklabels=list(inv_class_mapping.values()))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - {seg_folder} DenseNet121 model')

    # Save confusion matrix
    cm_save_path = os.path.join(SAVE_DIR, f'confusion_matrix_densenet_{seg_folder}.png')
    plt.savefig(cm_save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Confusion matrix saved to: {cm_save_path}")

    # Plot and save ROC AUC curve
    roc_auc_scores = plot_roc_auc_curve(y_test, y_pred, seg_folder, SAVE_DIR)

    # Plot and save training history
    plt.figure(figsize=(15, 5))

    # Accuracy plot
    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    plt.title(f'Model Accuracy - {seg_folder}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Loss plot
    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title(f'Model Loss - {seg_folder}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Learning rate plot (if available)
    plt.subplot(1, 3, 3)
    if 'lr' in history.history:
        plt.plot(history.history['lr'], label='Learning Rate', linewidth=2)
        plt.title(f'Learning Rate - {seg_folder}')
        plt.ylabel('Learning Rate')
        plt.xlabel('Epoch')
        plt.yscale('log')
        plt.legend()
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'Learning Rate\nNot Available',
                ha='center', va='center', transform=plt.gca().transAxes, fontsize=12)
        plt.title(f'Learning Rate - {seg_folder}')

    plt.tight_layout()

    # Save training history
    history_save_path = os.path.join(SAVE_DIR, f'training_history_densenet_{seg_folder}.png')
    plt.savefig(history_save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Training history saved to: {history_save_path}")

    # Save the model
    model_path = os.path.join(SAVE_DIR, f'densenet_classifier_{seg_folder}.keras')
    model.save(model_path)
    print(f"DenseNet121 Model saved to {model_path}")

    return model, X_test, y_test, y_test_cat, roc_auc_scores

# Enhanced ensemble predictions with ROC AUC
def create_ensemble_predictions(models, test_data):
    """Create ensemble predictions by averaging the outputs of individual models"""
    print("\n=== Creating Ensemble Predictions ===")

    # Use the first segmentation method's test set as our evaluation set
    first_folder = next(iter(test_data))
    X_test, y_test, y_test_cat = test_data[first_folder][:3]

    ensemble_predictions = []

    # Collect predictions from all models
    for folder, model in models.items():
        # Generate predictions
        y_pred = model.predict(X_test)
        ensemble_predictions.append(y_pred)

    # Average predictions
    avg_predictions = np.mean(ensemble_predictions, axis=0)
    ensemble_classes = np.argmax(avg_predictions, axis=1)

    # Evaluate ensemble performance
    from sklearn.metrics import accuracy_score
    ensemble_accuracy = accuracy_score(y_test, ensemble_classes)
    print(f"Ensemble DenseNet121 Model Accuracy: {ensemble_accuracy:.4f}")

    # Print classification report
    print("\nClassification Report for Ensemble DenseNet121 Model:")
    print(classification_report(y_test, ensemble_classes,
                              target_names=list(inv_class_mapping.values())))

    # Plot and save confusion matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, ensemble_classes)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=list(inv_class_mapping.values()),
               yticklabels=list(inv_class_mapping.values()))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - Ensemble DenseNet121 Model')

    # Save confusion matrix
    cm_save_path = os.path.join(SAVE_DIR, 'confusion_matrix_densenet_ensemble.png')
    plt.savefig(cm_save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Ensemble confusion matrix saved to: {cm_save_path}")

    # Plot and save ROC AUC curve for ensemble
    ensemble_roc_auc = plot_roc_auc_curve(y_test, avg_predictions, 'ensemble', SAVE_DIR)

    return ensemble_accuracy, ensemble_classes, ensemble_roc_auc

# FIXED multi-input model using DenseNet121 with enhanced saving
def create_multi_input_densenet(all_X_train, all_X_test, y_train_cat, y_test_cat):
    """Create an improved model that takes all segmentation types as input using DenseNet121"""
    print("\n=== Creating Multi-Input DenseNet121 Model ===")

    # FIXED: Verify input data sizes
    print("📊 Multi-input data verification:")
    for folder, X_data in all_X_train.items():
        print(f"   {folder} training data: {X_data.shape[0]} images")
    for folder, X_data in all_X_test.items():
        print(f"   {folder} testing data: {X_data.shape[0]} images")

    # Create input branches for each segmentation type
    input_layers = []
    feature_layers = []

    for i, folder in enumerate(all_X_train.keys()):
        # Input layer for this segmentation type
        input_img = Input(shape=(224, 224, 3), name=f'input_{folder}')
        input_layers.append(input_img)

        # Create a DenseNet121 base for each input with unique name
        base_model = DenseNet121(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3),
            name=f'densenet121_{folder}'  # FIXED: Add unique name for each DenseNet121
        )

        # Freeze most layers but allow fine-tuning of last few
        base_model.trainable = True
        for layer in base_model.layers[:-10]:
            layer.trainable = False

        # Extract features using DenseNet121
        features = base_model(input_img)
        features = GlobalAveragePooling2D(name=f'gap_{folder}')(features)  # Unique name
        features = BatchNormalization(name=f'bn1_{folder}')(features)  # Unique name
        features = Dense(256, activation='relu', name=f'dense1_{folder}')(features)  # Unique name
        features = Dropout(0.3, name=f'dropout1_{folder}')(features)  # Unique name

        feature_layers.append(features)

    # Combine branches
    if len(feature_layers) > 1:
        combined = concatenate(feature_layers, name='concatenate_features')
    else:
        combined = feature_layers[0]

    # Classification layers with unique names
    x = Dense(512, activation='relu', name='classification_dense1')(combined)
    x = BatchNormalization(name='classification_bn1')(x)
    x = Dropout(0.5, name='classification_dropout1')(x)
    x = Dense(256, activation='relu', name='classification_dense2')(x)
    x = Dropout(0.3, name='classification_dropout2')(x)
    outputs = Dense(3, activation='softmax', name='classification_output')(x)

    # Create model
    model = Model(inputs=input_layers, outputs=outputs, name='multi_input_densenet')

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.00001),  # Lower learning rate for fine-tuning
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Create callbacks
    model_checkpoint = ModelCheckpoint(
        os.path.join(SAVE_DIR, 'best_multi_input_densenet_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=0.000001
    )

    # Prepare input data for training
    train_inputs = [all_X_train[folder] for folder in all_X_train.keys()]
    test_inputs = [all_X_test[folder] for folder in all_X_train.keys()]

    # Train the model
    history = model.fit(
        train_inputs, y_train_cat,
        batch_size=8,  # Smaller batch size due to multiple DenseNet121 models
        epochs=40,
        validation_data=(test_inputs, y_test_cat),
        callbacks=[early_stopping, reduce_lr, model_checkpoint]
    )

    # Evaluate the model
    test_loss, test_acc = model.evaluate(test_inputs, y_test_cat)
    print(f"Multi-Input DenseNet121 Model - Test accuracy: {test_acc:.4f}")

    # Get predictions
    y_pred = model.predict(test_inputs)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test_cat, axis=1)

    # Print classification report
    print("\nClassification Report for Multi-Input DenseNet121 Model:")
    print(classification_report(y_test_classes, y_pred_classes,
                              target_names=list(inv_class_mapping.values())))

    # Plot and save confusion matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test_classes, y_pred_classes)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=list(inv_class_mapping.values()),
               yticklabels=list(inv_class_mapping.values()))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - Multi-Input DenseNet121 Model')

    # Save confusion matrix
    cm_save_path = os.path.join(SAVE_DIR, 'confusion_matrix_multi_input_densenet.png')
    plt.savefig(cm_save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Multi-input confusion matrix saved to: {cm_save_path}")

    # Plot and save ROC AUC curve
    multi_input_roc_auc = plot_roc_auc_curve(y_test_classes, y_pred, 'multi_input', SAVE_DIR)

    # Plot and save training history
    plt.figure(figsize=(15, 5))

    # Accuracy plot
    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    plt.title('Multi-Input DenseNet121 Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Loss plot
    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title('Multi-Input DenseNet121 Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Learning rate plot (if available)
    plt.subplot(1, 3, 3)
    if 'lr' in history.history:
        plt.plot(history.history['lr'], label='Learning Rate', linewidth=2)
        plt.title('Multi-Input DenseNet121 Learning Rate')
        plt.ylabel('Learning Rate')
        plt.xlabel('Epoch')
        plt.yscale('log')
        plt.legend()
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'Learning Rate\nNot Available',
                ha='center', va='center', transform=plt.gca().transAxes, fontsize=12)
        plt.title('Multi-Input DenseNet121 Learning Rate')

    plt.tight_layout()

    # Save training history
    history_save_path = os.path.join(SAVE_DIR, 'training_history_multi_input_densenet.png')
    plt.savefig(history_save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Multi-input training history saved to: {history_save_path}")

    # Save the model
    model_path = os.path.join(SAVE_DIR, 'multi_input_densenet_classifier.keras')
    model.save(model_path)
    print(f"Multi-Input DenseNet121 Model saved to {model_path}")

    return model, multi_input_roc_auc, test_acc

# Main execution
def main():
    """Main function to train all models and create ensemble"""
    print("🚀 Starting DenseNet121 Training Pipeline")
    print("=" * 50)

    models = {}
    test_data = {}
    all_roc_scores = {}

    # Train individual models for each segmentation method
    for seg_folder in segmentation_folders:
        print(f"\n{'='*60}")
        print(f"Processing {seg_folder} segmentation")
        print(f"{'='*60}")

        result = train_model_for_segmentation(seg_folder, epochs=30, fine_tune=True)

        if result[0] is not None:  # If model training was successful
            model, X_test, y_test, y_test_cat, roc_auc_scores = result
            models[seg_folder] = model
            test_data[seg_folder] = (X_test, y_test, y_test_cat)
            all_roc_scores[seg_folder] = roc_auc_scores
        else:
            print(f"❌ Failed to train model for {seg_folder}")

        # Force garbage collection after each model
        gc.collect()

    # Create ensemble predictions if we have multiple models
    if len(models) > 1:
        ensemble_accuracy, ensemble_classes, ensemble_roc_auc = create_ensemble_predictions(models, test_data)
        all_roc_scores['ensemble'] = ensemble_roc_auc
    else:
        print("⚠️  Not enough models for ensemble creation")

    # Create multi-input model if we have data from multiple segmentation methods
    if len(models) >= 2:
        print("\n" + "="*60)
        print("Creating Multi-Input DenseNet121 Model")
        print("="*60)

        # Prepare data for multi-input model
        all_X_train = {}
        all_X_test = {}

        # Use the same train/test split for all segmentation methods
        for seg_folder in models.keys():
            X, y, filenames = load_data_from_folder(seg_folder)
            if len(X) > 0:
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.2, random_state=42, stratify=y
                )

                # Preprocess data
                X_train = X_train.astype(np.float32) / 255.0
                X_test = X_test.astype(np.float32) / 255.0

                all_X_train[seg_folder] = X_train
                all_X_test[seg_folder] = X_test

        if len(all_X_train) >= 2:
            # Convert labels to categorical (using the last y_train, y_test from the loop)
            y_train_cat = to_categorical(y_train, num_classes=3)
            y_test_cat = to_categorical(y_test, num_classes=3)

            multi_input_model, multi_input_roc_auc, multi_input_acc = create_multi_input_densenet(
                all_X_train, all_X_test, y_train_cat, y_test_cat
            )
            all_roc_scores['multi_input'] = multi_input_roc_auc
        else:
            print("⚠️  Not enough segmentation methods for multi-input model")
    else:
        print("⚠️  Not enough models for multi-input model creation")

    # Print summary of all results
    print("\n" + "="*60)
    print("FINAL RESULTS SUMMARY")
    print("="*60)

    for method, roc_scores in all_roc_scores.items():
        if roc_scores:
            print(f"\n{method.upper()} Model ROC AUC Scores:")
            print(f"  Micro-average: {roc_scores.get('micro', 'N/A'):.4f}")
            print(f"  Macro-average: {roc_scores.get('macro', 'N/A'):.4f}")
            for i, class_name in enumerate(['FT', 'ST', 'TT']):
                if i in roc_scores:
                    print(f"  {class_name}: {roc_scores[i]:.4f}")

    print(f"\n✅ All results saved to: {SAVE_DIR}")
    print("🎉 DenseNet121 Training Pipeline Complete!")

if __name__ == "__main__":
    main()



Results will be saved to: /content/densenet_results
🚀 Starting DenseNet121 Training Pipeline

Processing max segmentation

=== Training DenseNet121 model for max segmentation ===
🔄 Loading 7130 unique images from max folder...
📋 Sample filenames:
   1. FT100_pred_bright.jpg
   2. FT100_pred_cropped.jpg
   3. FT100_pred_flipped.jpg
   4. FT100_pred_original.jpg
   5. FT100_pred_rotated.jpg
✅ Successfully loaded: 7130 unique images
📊 Class distribution in max:
   FT: 740 images
   ST: 1900 images
   TT: 4490 images
🔍 Total images loaded: 7130
📊 Data verification for max:
   Total images loaded: 7130
   Unique filenames: 7130
   Expected ~7130 images per folder
📊 Data split for max:
   Training images: 5704
   Testing images: 1426
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/30
357/357 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.4329 - loss: 1.2931
Epoch 1: val_accuracy improved from -inf to 0.78471, saving model to /content/densenet_results/best_densenet_model_max.keras